# Setup

In [1]:
import numpy as np

import seaborn as sns
import pandas as pd
import os
import pprint
import ast
import re
import matplotlib.pyplot as plt
import dill
import requests
import xmltodict
import json

pp = pprint.PrettyPrinter(depth=6)

os.chdir(os.path.expanduser('~/vivarium-ecoli'))

ALLOWED_METAL_NAMES =   {'Iron': 'FE+2', 'Cobalt': 'CO+2', 'Copper': 'CU+2', 'Manganese': 'MN+2', 'Molybdenum': 'CPD-8123', 'Nickel': 'NI+2', 'Zinc': 'ZN+2',
                        'Calcium': 'CA+2', 'Magnesium': 'MG+2', 'Sodium': 'NA+', 'Potassium': 'K+',
                        'Iron-sulfur \(4Fe-4S\)': 'CPD-7', 'Iron-sulfur \(2Fe-2S\)': 'CPD-6',
                         'Iron-sulfur \(4Fe-4S-S-AdoMet\)': 'CPD-7', 'Iron-sulfur \(3Fe-4S\)': '3FE-4S', 'Iron-oxo-sulfur \(4Fe-2O-2S\)': 'CPD-7',
                        'heme': 'Heme-b', 'Molybdate': 'CPD-3', 'heme B': 'Heme-b',
                         'Selenocysteine': 'L-SELENOCYSTEINE',
                        'Divalent metal cation': 'Any+2'}


ACCEPTED_OTHER_FEATURES = {'PYRIDOXAL_PHOSPHATE', 'THIAMINE-PYROPHOSPHATE', 'FMN', 'FAD'}

## Connect to api

In [2]:
password = input("Enter Password: ")

In [3]:
s = requests.Session() # create session
# Post login credentials to session:
s.post('https://websvc.biocyc.org/credentials/login/', data={'email':'cellulararchitect@protonmail.com', 'password': password})

<Response [200]>

## api test

In [4]:
# example entry
# entity = 'PWY0-1356'
entity = 'FDOG-MONOMER'
req_str = f'https://websvc.biocyc.org/getxml?id=ECOLI:{entity}&detail=high'

r = s.get(req_str)
if r.status_code != 200:
    print(entity, r.status_code)

o = xmltodict.parse(r.content)
pp.pprint(o['ptools-xml']['Protein']['has-feature'][5]['Feature']['parent']['Feature']['@frameid'])

'Selenocysteine-sites'


In [5]:
entity = 'Fermentation'

fn = 'get-class-all-instances'
req_str = f'https://websvc.biocyc.org/apixml?fn={fn}&id=ECOLI:{entity}&detail=none'

r = s.get(req_str)
if r.status_code != 200:
    print(entity, r.status_code)

o = xmltodict.parse(r.content)
pp.pprint(o['ptools-xml'])

{'@ptools-version': '27.0',
 '@xml:base': 'http://BioCyc.org/apixml?fn=get-class-all-instances%26id=ECOLI:Fermentation%26detail=NONE',
 'Pathway': [{'@frameid': 'PWY-5480',
              '@orgid': 'ECOLI',
              '@resource': 'getxml?ECOLI:PWY-5480'},
             {'@frameid': 'PWY0-1312',
              '@orgid': 'ECOLI',
              '@resource': 'getxml?ECOLI:PWY0-1312'},
             {'@frameid': 'PWY-5485',
              '@orgid': 'ECOLI',
              '@resource': 'getxml?ECOLI:PWY-5485'},
             {'@frameid': 'FERMENTATION-PWY',
              '@orgid': 'ECOLI',
              '@resource': 'getxml?ECOLI:FERMENTATION-PWY'},
             {'@frameid': 'PWY-5437',
              '@orgid': 'ECOLI',
              '@resource': 'getxml?ECOLI:PWY-5437'},
             {'@frameid': 'PWY-8274',
              '@orgid': 'ECOLI',
              '@resource': 'getxml?ECOLI:PWY-8274'}],
 'metadata': {'num_results': '6',
              'query': 'fn=get-class-all-instances&id=ECOLI:Fermenta

# Getting raw data

## Fetch EcoCyc pathway tree

In [6]:
def recursive_pathway_tree(current_node, parent_node, node_dict, level):


    if current_node in node_dict.keys():

        if level < node_dict[current_node]['level']:
            node_dict[current_node]['level'] = level

        node_dict[current_node]['parents'].append(parent_node)

        return

    else:

        req_str = f'https://websvc.biocyc.org/getxml?id=ECOLI:{current_node}&detail=high'

        node_dict[current_node] = {'parents': [], 'children': [], 'level': level}

        if parent_node is not None:
            node_dict[current_node]['parents'].append(parent_node)

        r = s.get(req_str)
        if r.status_code != 200:
            print(current_node, r.status_code)
            return

        o = xmltodict.parse(r.content)


        subclasses = o['ptools-xml']['Pathway']['subclass'] if 'subclass' in o['ptools-xml']['Pathway'] else []
        if type(subclasses) is dict:
            subclasses = [subclasses]

        instances = o['ptools-xml']['Pathway']['instance'] if 'instance' in o['ptools-xml']['Pathway'] else []
        if type(instances) is dict:
            instances = [instances]

        pathways = subclasses + instances

        # print(f'{current_node}: {len(subclasses)}, {len(instances)}')

        for pathway in pathways:
            pathway_id = pathway['Pathway']['@frameid']

            node_dict[current_node]['children'].append(pathway_id)
            recursive_pathway_tree(pathway_id, current_node, node_dict, level+1)


    return


def get_pathway_ith_level_parents(cur_pathway_idx, pathway_matrix, name_list, level_vector, level=2, parent_dict=None):

    if parent_dict is None:
        parent_dict = {}

    cur_pathway_level = level_vector[cur_pathway_idx]

    if cur_pathway_level == level:
        parent_dict[name_list[cur_pathway_idx]] = cur_pathway_level

    parent_slice = pathway_matrix[:, cur_pathway_idx]
    parent_idxs = np.where(parent_slice != 0)[0]


    for idx in parent_idxs:

        _ = get_pathway_ith_level_parents(idx, pathway_matrix, name_list, level_vector, level, parent_dict)

    return parent_dict



In [7]:
entity = 'Pathways'

fn = 'get-class-direct-subs'
req_str = f'https://websvc.biocyc.org/apixml?fn={fn}&id=ECOLI:{entity}&detail=none'

r = s.get(req_str)
o = xmltodict.parse(r.content)['ptools-xml']['Pathway']
top_level_pathways = [pathway['@frameid'] for pathway in o]

pathway_node_dict = {}

for top_node in top_level_pathways:

    recursive_pathway_tree(top_node, None, pathway_node_dict, level=1)

ARG+POLYAMINE-SYN 404


In [8]:
pathway_df = pd.DataFrame(pathway_node_dict).T.reset_index(names='id')
pathway_df

,id,parents,children,level
0,Signaling-Pathways,[],"[PWY0-1559, PWY0-1495, PWY0-1518, PWY0-1490, P...",1
1,PWY0-1559,[Signaling-Pathways],[],2
2,PWY0-1495,[Signaling-Pathways],[],2
3,PWY0-1518,[Signaling-Pathways],[],2
4,PWY0-1490,[Signaling-Pathways],[],2
...,...,...,...,...
1159,GLUTATHIONESYN-PWY,[Reductants],[],4
1160,Butanediol-Biosynthesis,[Other-biosynthesis],[],3
1161,CYCLOPEPTIDES,[Other-biosynthesis],[],3
1162,6-HM-Dihydropterin-PP-Biosynthesis,[Other-biosynthesis],[PWY-6147],3


## Fetch protein monomer annotations

In [9]:
# get a set of all monomers with an associated uniprot id
proteins_df = pd.read_csv('reconstruction/ecoli/flat/proteins.tsv', sep='\t').loc[:, ["id", "common_name"]]

for column in ["enzyme_reaction", "cofactors", "metal_features", "other_features", "direct_annotations"]:
    proteins_df[column] = 0
    proteins_df[column] = proteins_df[column].astype(object)

proteins_df

,id,common_name,enzyme_reaction,cofactors,metal_features,other_features,direct_annotations
0,1-ACYLGLYCEROL-3-P-ACYLTRANSFER-MONOMER,1-acylglycerol-3-phosphate <i>O</i>-acyltransf...,0,0,0,0,0
1,1-PFK-MONOMER,1-phosphofructokinase,0,0,0,0,0
2,2-DEHYDROPANTOATE-REDUCT-MONOMER,2-dehydropantoate 2-reductase,0,0,0,0,0
3,2-ISOPROPYLMALATESYN-MONOMER,2-isopropylmalate synthase,0,0,0,0,0
4,2-OCTAPRENYL-METHOXY-BENZOQ-METH-MONOMER,"bifunctional 2-octaprenyl-6-methoxy-1,4-benzoq...",0,0,0,0,0
...,...,...,...,...,...,...,...
4415,YTFR-MONOMER,galactofuranose ABC transporter putative ATP b...,0,0,0,0,0
4416,YTFT-MONOMER,galactofuranose ABC transporter putative membr...,0,0,0,0,0
4417,ZNUA-MONOMER,Zn<sup>2+</sup> ABC transporter periplasmic bi...,0,0,0,0,0
4418,ZNUB-MONOMER,Zn<sup>2+</sup> ABC transporter membrane subunit,0,0,0,0,0


In [10]:
for i in range(len(proteins_df.index)):
    if i % 100 == 0:
        print(i)

    protein = proteins_df.loc[i, 'id']
    proteins_df.at[i, 'other_features'] = set()
    proteins_df.at[i, 'metal_features'] = set()
    proteins_df.at[i, 'enzyme_reaction'] = set()
    proteins_df.at[i, 'cofactors'] = set()
    proteins_df.at[i, 'direct_annotations'] = set()

    req_str = f'https://websvc.biocyc.org/getxml?id=ECOLI:{protein}&detail=high'



    r = s.get(req_str)
    if r.status_code != 200:
        print(protein, r.status_code)
        continue

    o = xmltodict.parse(r.content)['ptools-xml']



    metal_set = set()
    other_feature_set = set()
    if 'Protein' in o and 'has-feature' in o['Protein']:
        features = o['Protein']['has-feature']

        if type(features) is dict:
            features = [features]

        for feature in features:
            if 'parent' not in feature['Feature']:
                continue

            category = feature['Feature']['parent']['Feature']['@frameid']
            if category == 'Metal-Binding-Sites' and 'comment' in feature['Feature']:

                # Detect match to any of the allowed metal names and allowed cofactor names and add to list
                comment = feature['Feature']['comment']['#text']
                metal_set.add(comment)

                proteins_df.at[i, 'metal_features'] = list(metal_set)

            if category == 'Nucleotide-Phosphate-Binding-Regions' and 'attached-group' in feature['Feature'] and 'Compound' in feature['Feature']['attached-group']:
                attached_group = feature['Feature']['attached-group']['Compound']['@frameid']
                other_feature_set.add(attached_group)

            if category == 'N6-pyridoxal-phosphate-Lys-Modifications':
                other_feature_set.add('PYRIDOXAL_PHOSPHATE')

            if category == 'Protein-Segments' and 'comment' in feature['Feature'] and 'Thiamine' in feature['Feature']['comment']['#text']:
                other_feature_set.add('THIAMINE-PYROPHOSPHATE')

            if category == 'Selenocysteine-sites':
                print('found selenocysteine')
                metal_set.add('Selenocysteine.')

            proteins_df.at[i, 'metal_features'] = list(metal_set)
            proteins_df.at[i, 'other_features'] = list(other_feature_set)

        if 'catalyzes' in o['Protein']:
            oc = o['Protein']['catalyzes']['Enzymatic-Reaction']

            if type(oc) is dict:
                oc = [oc]

            cofactor_set = set()
            enz_rxn_set = set()

            for enzrxn in oc:
                enz_id = enzrxn['@frameid']

                enz_rxn_set.add(enz_id)

                enz_req_str = f'https://websvc.biocyc.org/getxml?id=ECOLI:{enz_id}&detail=high'

                rz = s.get(enz_req_str)
                oe = xmltodict.parse(rz.content)['ptools-xml']['Enzymatic-Reaction']

                if "cofactor" in oe:
                    oe = oe['cofactor']

                    if type(oe) is dict:
                        oe = [oe]

                    for cofactor in oe:
                        cof = cofactor['Compound']['@frameid']
                        cofactor_set.add(cof)

            proteins_df.at[i, 'enzyme_reaction'] = enz_rxn_set
            proteins_df.at[i, 'cofactors'] = cofactor_set

    # get gene pathway annotation
    if 'Protein' in o and 'gene' in o['Protein']:

        gene = o['Protein']['gene']['Gene']['@frameid']
        gene_req_str = f'https://websvc.biocyc.org/apixml?fn=pathways-of-gene&id=ECOLI:{gene}&detail=none'

        rg = s.get(gene_req_str)
        og = xmltodict.parse(rg.content)['ptools-xml']

        if 'Pathway' in og:
            pathways = og['Pathway']

            if type(pathways) is dict:
                pathways = [pathways]

            for pathway in pathways:
                pathway_id = pathway['@frameid']
                proteins_df.at[i, 'direct_annotations'].add(pathway_id)




0
100
200
300
400
500
600
700
800
900
1000
1100
EG11708-MONOMER 404
1200
1300
1400
1500
1600
1700
found selenocysteine
found selenocysteine
found selenocysteine
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400


In [11]:
# find row with id FDOG-MONOMER
protein_idx = np.where(proteins_df['id'] == 'FDOG-MONOMER')[0][0]
proteins_df.at[protein_idx, 'metal_features']



['Selenocysteine.', 'UniProt: Iron-sulfur (4Fe-4S).']

In [12]:
# proteins_df.to_parquet('notebooks/fbagd/data/raw_protein_features.parquet', index=False)

## Reload protein feature table

In [13]:
# proteins_df = pd.read_parquet('notebooks/fbagd/data/raw_protein_features.parquet')
# proteins_df['metal_features'] = proteins_df['metal_features'].apply(ast.literal_eval)
# # for rows of proteins with where other_features is set, convert from string to set with literal_eval
# proteins_df.loc[proteins_df['other_features'].str.startswith('['), 'other_features'] = \
#     proteins_df.loc[proteins_df['other_features'].str.startswith('['), 'other_features'].apply(ast.literal_eval)

filter_protein_df = proteins_df.copy().loc[:, ['id', 'common_name', 'metal_features', 'other_features', 'enzyme_reaction', 'cofactors', 'direct_annotations']]
filter_protein_df

,id,common_name,metal_features,other_features,enzyme_reaction,cofactors,direct_annotations
0,1-ACYLGLYCEROL-3-P-ACYLTRANSFER-MONOMER,1-acylglycerol-3-phosphate <i>O</i>-acyltransf...,[],[],"{ENZRXN0-8629, ENZRXN0-6620, ENZRXN0-7995, ENZ...",{},"{PWY-5667, PWY0-1319}"
1,1-PFK-MONOMER,1-phosphofructokinase,[],[ATP],{},{},{PWY0-1314}
2,2-DEHYDROPANTOATE-REDUCT-MONOMER,2-dehydropantoate 2-reductase,[],[NADP],{2-DEHYDROPANTOATE-REDUCT-ENZRXN},{},{PANTO-PWY}
3,2-ISOPROPYLMALATESYN-MONOMER,2-isopropylmalate synthase,[],[],{ENZRXN0-6250},{},{LEUSYN-PWY}
4,2-OCTAPRENYL-METHOXY-BENZOQ-METH-MONOMER,"bifunctional 2-octaprenyl-6-methoxy-1,4-benzoq...",[],[],"{ADOMET-DMK-METHYLTRANSFER-ENZRXN, 2-OCTAPRENY...",{},"{MENAQUINONESYN-PWY, PWY-6708}"
...,...,...,...,...,...,...,...
4415,YTFR-MONOMER,galactofuranose ABC transporter putative ATP b...,[],[ATP],{},{},{}
4416,YTFT-MONOMER,galactofuranose ABC transporter putative membr...,[],[],{},{},{}
4417,ZNUA-MONOMER,Zn<sup>2+</sup> ABC transporter periplasmic bi...,"[implicated in a second zinc binding site, Uni...",[],{},{},{}
4418,ZNUB-MONOMER,Zn<sup>2+</sup> ABC transporter membrane subunit,[],[],{},{},{}


## Annotate complexation with EcoCyc data

In [14]:
complex_df = pd.read_csv('reconstruction/ecoli/flat/complexation_reactions.tsv', sep='\t').loc[:, ['id', 'stoichiometry']]


removed_complexes = pd.read_csv('reconstruction/ecoli/flat/complexation_reactions_removed.tsv', sep='\t')

# remove rows where id starts with '#'
complex_df = complex_df[~complex_df['id'].str.startswith('#')].reset_index(drop=True)

# remove rows of complex_df where id matches an id in removed_complexes
complex_df = complex_df[~complex_df['id'].isin(removed_complexes['id'])].reset_index(drop=True)
complex_df.stoichiometry = complex_df.stoichiometry.astype(object)


for i, stoich in enumerate(complex_df.loc[:, 'stoichiometry']):

    if type(stoich) is str and stoich[0] == '{':
        stoich = stoich.replace('null', '-1')
        stoich = ast.literal_eval(stoich)

        complex_df.at[i, 'stoichiometry'] = stoich

    else:
        complex_df.at[i, 'stoichiometry'] = {}


# for each row, find dict entry with positive value
for i in range(len(complex_df.index)):

    stoich = complex_df.loc[i, 'stoichiometry']

    for k,v in stoich.items():
        if v > 0:
            complex_df.at[i, 'id'] = k

complex_df

,id,stoichiometry
0,1-PFK,"{'1-PFK': 1, '1-PFK-MONOMER': -2}"
1,2OXOGLUTARATEDEH-CPLX,"{'2OXOGLUTARATEDEH-CPLX': 1, 'E1O': -1, 'E2O':..."
2,3-ISOPROPYLMALDEHYDROG-CPLX,"{'3-ISOPROPYLMALDEHYDROG-CPLX': 1, '3-ISOPROPY..."
3,3-ISOPROPYLMALISOM-CPLX,"{'3-ISOPROPYLMALISOM-CPLX': 1, 'LEUC-MONOMER':..."
4,3-METHYL-2-OXOBUT-OHCH3XFER-CPLX,"{'3-METHYL-2-OXOBUT-OHCH3XFER-CPLX': 1, '3-CH3..."
...,...,...
1063,CPLX0-8053,"{'CPLX0-8053': 1, 'EG10942-MONOMER': -1}"
1064,CPLX0-8253,"{'CPLX0-8253': 1, 'CSRC-RNA': -1, 'EG11447-MON..."
1065,SRP-CPLX,"{'SRP-CPLX': 1, 'EG10300-MONOMER': -1, 'FFS-RN..."
1066,CPLX0-7796APO,"{'CPLX0-7796APO': 1, 'PD04032': -2}"


In [15]:
complex_df["cofactors"] = 0
complex_df["cofactors"] = complex_df["cofactors"].astype(object)

complex_df["enzyme_reaction"] = 0
complex_df["enzyme_reaction"] = complex_df["enzyme_reaction"].astype(object)

for i in range(len(complex_df.index)):

    if i % 100 == 0:
        print(i)

    complex = complex_df.loc[i, 'id']

    req_str = f'https://websvc.biocyc.org/getxml?id=ECOLI:{complex}&detail=low'

    r = s.get(req_str)
    if r.status_code != 200:
        print(complex, r.status_code)
        complex_df.at[i, 'enzyme_reaction'] = set()
        complex_df.at[i, 'cofactors'] = set()
        continue

    o = xmltodict.parse(r.content)['ptools-xml']['Protein']


    # if enzyme
    if 'catalyzes' in o:
        o = o['catalyzes']['Enzymatic-Reaction']

        if type(o) is dict:
            o = [o]

        cofactor_set = set()
        enz_rxn_set = set()

        for enzrxn in o:
            enz_id = enzrxn['@frameid']

            enz_rxn_set.add(enz_id)

            enz_req_str = f'https://websvc.biocyc.org/getxml?id=ECOLI:{enz_id}&detail=high'

            rz = s.get(enz_req_str)
            oe = xmltodict.parse(rz.content)['ptools-xml']['Enzymatic-Reaction']

            if "cofactor" in oe:
                oe = oe['cofactor']

                if type(oe) is dict:
                    oe = [oe]

                for cofactor in oe:
                    cof = cofactor['Compound']['@frameid']
                    cofactor_set.add(cof)

        complex_df.at[i, 'enzyme_reaction'] = enz_rxn_set
        complex_df.at[i, 'cofactors'] = cofactor_set

    else:
        complex_df.at[i, 'enzyme_reaction'] = set()
        complex_df.at[i, 'cofactors'] = set()

0
100
200
CPLX0-2423 404
300
CPLX0-3976 404
400
500
600
700
800
900
RECFOR-CPLX 404
1000
CPLX0-7796APO 404


## Get cofactor elemental composition and more

In [16]:
# get set of all cofactors
all_metal_cofactors = set(ALLOWED_METAL_NAMES.values())
all_other_cofactors = ACCEPTED_OTHER_FEATURES

filter_cofactor_df = pd.DataFrame(columns=['id', 'elemental_composition'])
filter_cofactor_df['id'] = list(all_metal_cofactors | all_other_cofactors)

filter_cofactor_df['elemental_composition'] = 0
filter_cofactor_df['elemental_composition'] = filter_cofactor_df['elemental_composition'].astype(object)

# for each cofactor, get elemental composition
for i in range(len(filter_cofactor_df.index)):

    compound = filter_cofactor_df.loc[i, 'id']
    atom_dict = {}

    url_name = compound.replace('+', '%2b')

    req_str = f'https://websvc.biocyc.org/getxml?id=ECOLI:{url_name}&detail=full'

    r = s.get(req_str)
    if r.status_code != 200:
        print(compound, r.status_code)
        filter_cofactor_df.at[i, 'elemental_composition'] = {}
        continue

    o = xmltodict.parse(r.content)['ptools-xml']

    atoms = o['Compound']['cml']['molecule']['atomArray']['atom']
    if type(atoms) is dict:
        atoms = [atoms]

    for atom in atoms:
        element = atom['@elementType']

        # either add new element or add to existing element
        if element not in atom_dict:
            atom_dict[element] = 1
        else:
            atom_dict[element] += 1

    filter_cofactor_df.at[i, 'elemental_composition'] = atom_dict

Any+2 404


In [17]:
filter_cofactor_df

,id,elemental_composition
0,FAD,"{'C': 27, 'N': 9, 'O': 15, 'P': 2}"
1,Heme-b,"{'C': 34, 'N': 4, 'O': 4, 'FE': 1}"
2,FE+2,{'FE': 1}
3,NI+2,{'NI': 1}
4,CA+2,{'CA': 1}
5,L-SELENOCYSTEINE,"{'C': 3, 'N': 1, 'O': 2, 'SE': 1}"
6,CPD-7,"{'FE': 4, 'S': 4}"
7,MN+2,{'MN': 1}
8,3FE-4S,"{'FE': 3, 'S': 4}"
9,ZN+2,{'ZN': 1}


## Save tables to avoid having to re-download

In [18]:
# save complex_df to csv in a way that preserves dicts
complex_df.to_csv('notebooks/cofactors/data/raw_complexes.csv', index=False)
proteins_df.to_csv('notebooks/cofactors/data/raw_proteins.csv', index=False)
filter_cofactor_df.to_csv('notebooks/cofactors/data/raw_cofactors.csv', index=False)
pathway_df.to_csv('notebooks/cofactors/data/raw_pathways.csv', index=False)

In [19]:
complex_df

,id,stoichiometry,cofactors,enzyme_reaction
0,1-PFK,"{'1-PFK': 1, '1-PFK-MONOMER': -2}",{MG+2},{1PFRUCTPHOSPHN-ENZRXN}
1,2OXOGLUTARATEDEH-CPLX,"{'2OXOGLUTARATEDEH-CPLX': 1, 'E1O': -1, 'E2O':...","{FAD, MG+2, LIPOIC-ACID, THIAMINE-PYROPHOSPHATE}",{2OXOGLUTARATEDEH-ENZRXN}
2,3-ISOPROPYLMALDEHYDROG-CPLX,"{'3-ISOPROPYLMALDEHYDROG-CPLX': 1, '3-ISOPROPY...","{MN+2, MG+2}",{3-ISOPROPYLMALDEHYDROG-ENZRXN}
3,3-ISOPROPYLMALISOM-CPLX,"{'3-ISOPROPYLMALISOM-CPLX': 1, 'LEUC-MONOMER':...",{CPD-7},{3-ISOPROPYLMALISOM-ENZRXN}
4,3-METHYL-2-OXOBUT-OHCH3XFER-CPLX,"{'3-METHYL-2-OXOBUT-OHCH3XFER-CPLX': 1, '3-CH3...",{MG+2},{3-METHYL-2-OXOBUT-OHCH3XFER-ENZRXN}
...,...,...,...,...
1063,CPLX0-8053,"{'CPLX0-8053': 1, 'EG10942-MONOMER': -1}",{},{}
1064,CPLX0-8253,"{'CPLX0-8253': 1, 'CSRC-RNA': -1, 'EG11447-MON...",{},{}
1065,SRP-CPLX,"{'SRP-CPLX': 1, 'EG10300-MONOMER': -1, 'FFS-RN...",{},{}
1066,CPLX0-7796APO,"{'CPLX0-7796APO': 1, 'PD04032': -2}",{},{}


In [20]:

# if 'Protein' in o and 'has-go-term' in o['Protein']:
#     go_terms = o['Protein']['has-go-term']
#
#     annotation_set = set()
#
#     if type(go_terms) is dict:
#         go_terms = [go_terms]
#
#     for go_term in go_terms:
#         # example entry
#         entity = go_term['GO-Term']['@frameid']
#         entity = entity.replace(':', '%3A')
#         req_str = f'https://www.ebi.ac.uk/QuickGO/services/ontology/go/terms/{entity}'
#
#         r = s.get(req_str)
#         if r.status_code != 200:
#             print(entity, r.status_code)
#
#         o = json.loads(r.content)
#         if len(o['results']) != 1:
#             print('error', entity, len(o['results']))
#             continue
#         elif o['results'][0]['aspect'] == 'biological_process':
#             annotation_set.add(o['results'][0]['name'])
#
#             o['ptools-xml']['Protein']['gene']['Gene']['@frameid']
#
#     proteins_df.at[i, 'direct_annotations'] = annotation_set